### References 
* [Pew Research Center: U.S Media Polarization and the 2020 Election](https://www.journalism.org/2020/01/24/americans-are-divided-by-party-in-the-sources-they-turn-to-for-political-news/)
* [Description of NEWS Ngram 2.0 Dataset](https://blog.gdeltproject.org/announcing-the-television-news-ngram-2-0-dataset/)
* Query used to collect data from BigQuery unigram table
>```SELECT *  FROM `gdelt-bq.gdeltv2.iatv_1gramsv2` WHERE DATE(TIMESTAMP) = "2020-11-03"```
* [Client Library Tutorial](https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries)
* [BigQuery Python API Documentation](https://cloud.google.com/bigquery/docs/authentication/service-account-file)

In [1]:
assert 1


In [2]:
import pathlib 
import datetime
import os
import re 
import time

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from google.cloud import bigquery
from google.oauth2 import service_account

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vignesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Ideas for what can be implemented 
1. Graph which shows change of words over time 


# A Walkthrough of 2020 Using Natual Language Processing 

1. Select all data since the beginning of 2020 using:
> ````SELECT *  FROM `gdelt-bq.gdeltv2.iatv_1gramsv2` WHERE DATE(TIMESTAMP) >= "2020-01-01"```
    * Query returns > 23 GB of data. Too large to be stored as a single file in Google Drive (1 GB limit)

* Goal of the project is to create an interactive calander which can be used to view how the media portrayed 2020
1. Collect data as monthly csv file

In [ ]:
today = datetime.datetime.today()
today


In [42]:
# utility functions 

# list all directory items 
def list_dir_items(filePath):
    return [x for x in os.listdir(filePath)]

def find_last_data_dump_date(listOfFileNames):
    pattern = re.compile('.*DataDump_(\d+).csv')
    arr = [int(pattern.findall(fileName)[0]) for fileName in listOfFileNames]
    return np.max(arr)


def construct_query(startTimeStamp, endTimeStamp=None):
    QUERY = (f'''
                 SELECT * 
                 FROM `gdelt-bq.gdeltv2.iatv_1gramsv2` 
                 WHERE 
                 (TIMESTAMP >= '{startTimeStamp}') AND 
                 (TIMESTAMP < '{endTimeStamp}')
             ''')
    return QUERY 
    
    
def execute_query(QUERY, startTimeStamp, endTimeStamp=None):
    # authenticate account
    key_path = "credentials.json"
    credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
        )
    
    # constuct BiQuery client object
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    
    print(f"\nExecuting query for {startTimeStamp.strftime('%Y-%m-%d')}: \n{QUERY}")
    
    # excecute query and return the RowIterator object
    job = client.query(QUERY)
    return job.result()

def save_rows(rows, startTimeStamp, dataDumpRoot):
    collector = []
    startTime = time.time()
    #TODO: Implement a time out functionality if looping through the RowIterator takes longer than a pre-specified time (say 5 mins/300 seconds)
    
    startTimeLoop = time.time()
    for n, row in enumerate(rows):
        currentTimeLoop  = time.time()
        if (currentTimeLoop - startTimeLoop) > 300: # raise an error if looping takes more than 5 minutes
          raise TimeoutError
        value = list(row.values())# convert to list since tuple is immutable
        # convert row values to comma seperates string and append to the collector
        value[0] = value[0].strftime('%Y-%m-%d %H-%M-%S')
        value = [str(x) for x in value]
        valueOut = ', '.join(value)
        valueOut = valueOut + '\n'
        collector.append(valueOut)
        
    # save results to data dump directory
    today = datetime.datetime.now().strftime('%y%m%d') 
    queryDate = startTimeStamp.strftime('%y%m%d') 
    outFileName = f"{today}_NewsNGram2_DataDump_{queryDate}.csv"
    outFilePath = pathlib.Path(dataDumpRoot, outFileName)
    with open(outFilePath, 'w', encoding="utf-8") as file:
        file.writelines(collector)
        
    # print how long it took to write the lines
    timeTaken = time.time() - startTime
    print(f"Completed writing {len(collector)} rows for {startTimeStamp.strftime('%Y-%m-%d')} in {timeTaken} seconds.")
    
    return collector
          
def construct_execute_save_query(startTimeStamp, endTimeStamp=None, dataDumpRoot=None):
    if not endTimeStamp: # by default, endTimeStamp is startTimeStamp + 24 hours 
            endTimeStamp = startTimeStamp + datetime.timedelta(days=1)
          
    QUERY = construct_query(startTimeStamp, endTimeStamp)
    print(f"{construct_query.__name__} successful")
    rows = execute_query(QUERY, startTimeStamp, endTimeStamp=endTimeStamp)
    print(f"{execute_query.__name__} successful")
    save_rows(rows, startTimeStamp, dataDumpRoot)
    print(f"{save_rows.__name__} successful")
    print('-'*100)

datetime.datetime(2020, 11, 19, 0, 15, 28, 65713)

In [37]:
datetime.datetime.now()==datetime.date.today()

False

In [50]:
# new plan 
# check root for a directory called 2020_DataDump 
    # if it not present make the directory
# dumpFileNameTemplate = 'TodaysDate_NewsNGram2_QueryDate.csv'
# eg: 201117_NewsNGram2_DataDump_200101.csv
def main(dataDumpRoot='2020_NewsNGram_DataDump', queryTimeStamp=None, queryTimeStampEnd=None, queryDataToDate=False):
    # list all files in the directory, find correct data to start qurying
    # if no files are present the datetime generator will start with the first day of the year 
    # else find the last day for which data is availble and start the generator from the next day
    if dataDumpRoot not in list_dir_items('./'):
        os.mkdir(dataDumpRoot)
        
    # if query timestamp is not provided by the user, constuct it
    if not queryTimeStamp:
        dataDumpRootDirItems = list_dir_items(dataDumpRoot)
        # start on 2020-01-01 if dataDumpRootDirItems is empty, else find the last day in the DataDump folder
        if not dataDumpRootDirItems:
            queryTimeStamp = datetime.datetime(year=2020, month=1, day=1)
        else:
            queryTimeStamp = find_last_data_dump_date(dataDumpRootDirItems)
            queryTimeStamp = datetime.datetime.strptime(str(queryTimeStamp), '%y%m%d')
            # if the day is not today, increment the day with data dump file to get the next day to query
            if queryTimeStamp.date() < datetime.date.today():
                queryTimeStamp += datetime.timedelta(days=1)
    else:
        queryTimeStamp = datetime.datetime.strptime(str(queryTimeStamp), '%Y-%m-%d')
        if  queryTimeStamp.date() > datetime.date.today():
                print('Script terminated!!')
                print(f'queryTimeStamp:{queryTimeStamp} cannot be in the future')
                return

    
    # if queryDataToDate is True, all data will be collected from queryTimeStamp to today
    if not queryDataToDate:
        # construct and execute the query -> returns a RowIterator object
        construct_execute_save_query(queryTimeStamp, queryTimeStampEnd, dataDumpRoot)
    else:
        while True:
            try:
                construct_execute_save_query(queryTimeStamp, queryTimeStampEnd, dataDumpRoot)
                # increment the day by one
                queryTimeStamp += datetime.timedelta(days=1)
                # continue the operation until today's date
                today = datetime.datetime.now()
                if queryTimeStamp > today:
                    break
            except:
                print(f"Error occured")
                print(f"Re-attempting {queryTimeStamp.strftime('%Y-%m-%d')}")
                      
            

    # iterate through results
        
#     return job
        
# iterate through the results, append to collector and dump the collected data as a CSV file into 2020_DataDump

In [ ]:
main(queryDataToDate=...)

construct_query successful

Executing query for 2020-11-19: 

                 SELECT * 
                 FROM `gdelt-bq.gdeltv2.iatv_1gramsv2` 
                 WHERE 
                 (TIMESTAMP >= '2020-11-19 00:00:00') AND 
                 (TIMESTAMP < '2020-11-20 00:00:00')
             
execute_query successful
Error occured
Re-attempting 2020-11-19
construct_query successful

Executing query for 2020-11-19: 

                 SELECT * 
                 FROM `gdelt-bq.gdeltv2.iatv_1gramsv2` 
                 WHERE 
                 (TIMESTAMP >= '2020-11-19 00:00:00') AND 
                 (TIMESTAMP < '2020-11-20 00:00:00')
             
execute_query successful
Error occured
Re-attempting 2020-11-19
construct_query successful

Executing query for 2020-11-19: 

                 SELECT * 
                 FROM `gdelt-bq.gdeltv2.iatv_1gramsv2` 
                 WHERE 
                 (TIMESTAMP >= '2020-11-19 00:00:00') AND 
                 (TIMESTAMP < '2020-11-20 00:00:00')
 

In [ ]:
#TODO Separately 
# 200424